In [100]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last_expr"

In [101]:
def scale_bounds(b):
    if b[1] - b[0] > 10:
        b = np.log10(b)
    c1i = (b[1] + b[0])*.5
    c2i = (b[1] - b[0])*.5
    return ((b[0] - c1i)/c2i, (b[1] - c1i)/c2i)
def unscale(p, b): 
    if b[1] - b[0] > 10:
        b = np.log10(b)
        c1i = (b[1] + b[0])*.5
        c2i = (b[1] - b[0])*.5        
        return 10**(p*c2i+c1i)
    c1i = (b[1] + b[0])*.5
    c2i = (b[1] - b[0])*.5
    return p*c2i+c1i

In [106]:
fs = 103e3
Ts = 1/fs 
times = np.arange(0, 4*Ts, 4*Ts/200)
tau = Ts 
Cmin = 1e-12
Cmax = 1e-6
Rmin = 10e2
Rmax = 10e3
Vmin = .1
Vmax = 10 

def voltage_response(t, R, C, V0):
    return V0 * (1 - np.exp(-t / (R * C)))

def cost_function(params, t, data):
    #need to unscale the params 
    R, C, V0 = params
    R = unscale(R, (Rmin, Rmax))
    C = unscale(C, (Cmin, Cmax))
    V0 = unscale(V0, (Vmin, Vmax))
    residuals = data - voltage_response(t, R, C, V0)
    return np.sum(residuals**2)



In [108]:

#subs = {R2: 10e3, R1:10e3, Vm:10, C1:.3e-9, w: 2*pi*fs}
# Initial guess for the parameters
#initial_guess = [50e3, 1.1e-10, 1]
initial_guess = [0, 0, 0]
# Observed data
t_data = np.array(times)  # times

V_data = 3.1*(1 - np.exp(-times / (tau)))   # observed voltages

result = minimize(cost_function, initial_guess, args=(t_data, V_data), bounds=([-1,1], [-1,1],[-1,1]))

estimated_R, estimated_C, estimated_V0 = result.x
print(f'R = {unscale(estimated_R, (Rmin, Rmax))}')
print(f'C = {unscale(estimated_C, (Cmin, Cmax))}')
print(f'V0 = {unscale(estimated_V0, (Vmin, Vmax))}')

R = 4260.01626014016
C = 2.2790280672200176e-09
V0 = 3.0999965565812753


In [109]:
result 

      fun: 6.495983779564482e-10
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-3.47725152e-04, -9.55072505e-03,  1.59355837e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 272
      nit: 23
     njev: 68
   status: 0
  success: True
        x: array([ 0.25882251, -0.99544394, -0.39394009])

In [9]:
result.x[0]*result.x[1]

1e-05

Realization - the optimize function returns a solution - but there were other solutions. 
With 3 unknowns, if I pick 1, I could plot the last two with respect to eachother. Or, A 3D plot showing the 
relationship of the last one as a function of two. 
I have not utilized constraints yet in optimation.  THe cost function above uses a known response and compares it to the model. 
Constraints are relationships amongst the parameters being optimized for, like R1*C2 = x. 

KVL and KCL are a kind of constraint too, but unless the parameters of the cost function are the branch voltages and currents, they can't be used in ```optimize``` Contraints are the circuit model equations (Tableau or MNA) because the topology is given or known.

The desired output (branch current or voltage) is some function of time that can be sketched or written down. If there is more than one desired output equation, use a combined wieghted cost function. Phase/initial conditions will effect convergence. 
Optimization won't leave degrees of freedom but will solve it entirely. 
It'll be really difficult to plot things symbollically. A chicken and egg scenario arises. It would be nice to be able to write down the circuit equations and then infer time of frequency domain behavior in symbolic terms - but
